In [ ]:
import pandas as pd
import sqlite3 as db
import numpy as np
import pymc3 as pm
import theano
floatX = theano.config.floatX
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import theano.tensor as T
import pickle

https://www.chrisstucchio.com/blog/2017/bayesian_linear_regression.html

https://docs.pymc.io/notebooks/multilevel_modeling.html

https://towardsdatascience.com/bayesian-linear-regression-in-python-using-machine-learning-to-predict-student-grades-part-2-b72059a8ac7e

https://docs.pymc.io/notebooks/GLM-hierarchical.html

https://docs.pymc.io/notebooks/multilevel_modeling.html

https://docs.pymc.io/notebooks/getting_started

https://github.com/WillKoehrsen/Data-Analysis/blob/master/bayesian_lr/Bayesian%20Linear%20Regression%20Project.ipynb

https://docs.pymc.io/notebooks/GLM.html

https://docs.pymc.io/notebooks/bayesian_neural_network_advi.html

http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf

https://towardsdatascience.com/markov-chain-monte-carlo-in-python-44f7e609be98

http://edwardlib.org

https://stats.stackexchange.com/questions/252577/bayes-regression-how-is-it-done-in-comparison-to-standard-regression

Bmik12#4%

https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations

https://docs.pymc.io/notebooks/rugby_analytics.html

https://docs.pymc.io/notebooks/PyMC3_tips_and_heuristic.html

https://dsaber.com/2016/08/27/analyze-your-experiment-with-a-multilevel-logistic-regression-using-pymc3%E2%80%8B/

https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations

https://docs.pymc.io/notebooks/GLM-logistic.html

https://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/

In [ ]:
X_Corr = X.corr()['REPUBLICAN_PCT'].reset_index()
X_Corr_list = X_Corr[(X_Corr['index'] != 'REPUBLICAN_PCT') 
                     & (abs(X_Corr['REPUBLICAN_PCT']) > 0.10)]['index'].tolist()

In [ ]:
RGN_TRACE = {}

for rgn in X['RGN_DESC'].unique():
    X_RGN = X[X['RGN_DESC'] == rgn]
    df12 = X_RGN[X_RGN['YR'] == 2012]
    df12.drop(columns = ['RGN_DESC','YR'], inplace = True)
    
    df16 = X_RGN[X_RGN['YR'] == 2016]
    df16.drop(columns = ['RGN_DESC','YR'], inplace = True)
    
    X_train, X_test, y_train, y_test = train_test_split(df16, df16['REPUBLICAN_PCT'], test_size = 0.1, random_state=42)
    
    formula = "REPUBLICAN_PCT ~ " + " + ".join(list(df12)[1:])
    with pm.Model() as model:
        #set variable means and sd
        grp_mean = pm.Normal('grp_mean', mu= 50, sigma=10)
        grp_sd = pm.Uniform('grp_sd', 0, 10)
        
        # The prior for the data likelihood is a Normal REPUBLICAN_PCT
        #priors = {'Intercept': pm.Normal.dist(mu=df12['REPUBLICAN_PCT'].mean(), 
        #                                    sigma=df12['REPUBLICAN_PCT'].std())}
        #for col in list(df12)[1:]:
       #     priors[col] =  pm.Normal.dist(mu=grp_mean, sigma=grp_sd)

        # Creating the model requires a formula and data (and optionally a family)
        pm.GLM.from_formula(formula, data = X_train, family = pm.glm.families.Normal())# priors = priors,#)

        # Perform Markov Chain Monte Carlo sampling letting PyMC3 choose the algorithm
        trace = pm.sample(2000, step = pm.NUTS(), cores = 1)

        model_formula = 'REPUBLICAN_PCT = '
        for variable in trace.varnames:
            model_formula += ' %0.2f * %s +' % (np.mean(trace[variable]), variable)
        
    RGN_TRACE[rgn] = trace
    print(model_formula)
    evaluate_trace(trace, X_train, X_test, y_train, y_test)
    break

In [ ]:
with pm.Model() as model:
    
    # Priors
    mu_a = pm.Normal('mu_a', mu=50., sigma=1e5)
    sigma_a = pm.HalfCauchy('sigma_a', 5)
    mu_b = pm.Normal('mu_b', mu=0., sigma=1e5)
    sigma_b = pm.HalfCauchy('sigma_b', 5)

    # Random intercepts
    a = pm.Normal('intercept', mu= mu_a, sigma=sigma_a, shape = len(regions['i']))
    
    # Random slopes
    beta = pm.Normal('beta', mu=mu_b, sigma=sigma_b, shape=len(regions['i']))
    
    #Regional random effects
    #theta = pm.Normal('theta', mu=0.0, tau=sd_att, shape=N)
    #mu_phi = CAR2('mu_phi', w=wmat2, a=amat2, tau=tau_c, shape=N)
        
    #lm_input = pm.Data('lm_input', X_train)
    #beta = pm.Normal("beta", mu=0, sigma=sd_att, shape=(len(list(X_train)), len(regions['i'])))
    
    #mu = pm.Deterministic('mu', tt.exp(tt.dot(X_train.values, beta[region]) + intercept[region]))
        # Regional random effects
    #theta = pm.Normal('theta', mu=0.0, tau=sd_att, shape=N)
    #mu_phi = CAR2('mu_phi', w=wmat2, a=amat2, tau=tau_c, shape=N)
    lm_input = pm.Data('lm_input', X_train)
    beta = pm.Normal("mu_rgn", mu=0, sigma=sd_att, shape=(len(list(X_train)), len(regions['i'])))
    
    mu = pm.Deterministic('mu', tt.exp(tt.dot(X_train.values, beta[region]) + a[region]))
    
    # Creating the model requires a formula and data (and optionally a family)
    likelihood = pm.Normal('y', mu=mu.ravel(), sd=sigma, observed=y_train.values)

    # Perform Markov Chain Monte Carlo sampling letting PyMC3 choose the algorithm
    trace = pm.sample(2000, step = pm.NUTS(), cores = 1)

    #model_formula = 'REPUBLICAN_PCT = '
    #for variable in trace.varnames:
    #    model_formula += ' %0.2f * %s +' % (np.mean(trace[variable]), variable)

In [ ]:
def construct_nn(X, y):
    n_hidden = 15

    # Initialize random weights between each layer
    init_1 = np.random.randn(X.shape[1], n_hidden).astype(floatX)
    init_2 = np.random.randn(n_hidden, n_hidden).astype(floatX)
    init_out = np.random.randn(n_hidden).astype(floatX)

    with pm.Model() as neural_network:
        # Trick: Turn inputs and outputs into shared variables using the data container pm.Data
        # It's still the same thing, but we can later change the values of the shared variable
        # (to switch in the test-data later) and pymc3 will just use the new data.
        # Kind-of like a pointer we can redirect.
        # For more info, see: http://deeplearning.net/software/theano/library/compile/shared.html
        ann_input = pm.Data('ann_input', X)
        ann_output = pm.Data('ann_output', y)

        # Weights from input to hidden layer
        weights_in_1 = pm.Normal('w_in_1', 0, sigma=1,
                                 shape=(X.shape[1], n_hidden),
                                 testval=init_1)

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal('w_1_2', 0, sigma=1,
                                shape=(n_hidden, n_hidden),
                                testval=init_2)

        # Weights from hidden layer to output
        weights_2_out = pm.Normal('w_2_out', 0, sigma=1,
                                  shape=(n_hidden,),
                                  testval=init_out)

        # Build neural-network using tanh activation function
        act_1 = pm.math.tanh(pm.math.dot(ann_input,
                                         weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1,
                                         weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2,
                                              weights_2_out))

        # Binary classification -> Bernoulli likelihood
        out = pm.Bernoulli('out',
                           act_out,
                           observed=ann_output,
                           total_size=y.shape[0] # IMPORTANT for minibatches
                          )
    return neural_network

In [ ]:
x = T.matrix('X')
# symbolic number of samples is supported, we build vectorized posterior on the fly
n = T.iscalar('n')
# Do not forget test_values or set theano.config.compute_test_value = 'off'

x.tag.test_value = np.empty_like(X_train[:10])
n.tag.test_value = 100
_sample_proba = approx.sample_node(neural_network.out.distribution.p,
                                   size=n,
                                   more_replacements={neural_network['ann_input']: x})
# It is time to compile the function
# No updates are needed for Approximation random generator
# Efficient vectorized form of sampling is used
sample_proba = theano.function([x, n], _sample_proba)

pred = sample_proba(X_test, 500).mean(0) > 0.5

In [ ]:
neural_network = construct_nn(X_train, y_train)

with neural_network:
    inference = pm.ADVI()
    approx = pm.fit(n=30000, method=inference)

    trace = approx.sample(draws=5000)

In [ ]:
with open(REPORT_PATH + 'bayesian_model.pickle', 'wb') as handle:
    pickle.dump(RGN_TRACE, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(REPORT_PATH + 'bayesian_model.pickle', 'rb') as handle:
    RGN_TRACE = pickle.load(handle)